## 이 노트북은 llms.txt 파일을 읽어서 Databricks Vector Search를 구성하는 예시입니다.

- llms.txt 위치 : https://docs.langchain.com/llms.txt
- LangChain 공식 문서(Docs by LangChain)를 벡터 검색 인덱스로 구축하고, Agent를 통해 질의합니다.

In [2]:
# ============================================================
# 패키지 설치 — 환경에 맞게 하나만 주석 해제하세요
# ============================================================

# --- [로컬 환경] ---
!uv add databricks-vectorsearch databricks-sdk "databricks-connect==17.3.*" langchain langchain-text-splitters requests pandas

# --- [Databricks 노트북 환경] ---
# %pip install databricks-vectorsearch databricks-sdk langchain langchain-text-splitters requests
# dbutils.library.restartPython()


Resolved 105 packages in 432ms                                       
⠙ Preparing packages... (0/2)                                                   ⠋ Preparing packages... (0/0)                                                   
⠙ Preparing packages... (0/2)-------------------     0 B/101.25 KiB          
⠙ Preparing packages... (0/2)-------------------     0 B/101.25 KiB          
tenacity             ------------------------------     0 B/28.25 KiB
⠙ Preparing packages... (0/2)-------------------     0 B/101.25 KiB          
tenacity             ------------------------------     0 B/28.25 KiB
⠙ Preparing packages... (0/2)------------------- 14.88 KiB/101.25 KiB        
tenacity             ------------------------------ 16.00 KiB/28.25 KiB
⠙ Preparing packages... (0/2)------------------- 14.88 KiB/101.25 KiB        
tenacity             ------------------------------ 16.00 KiB/28.25 KiB
⠙ Preparing packages... (0/2)------------------- 30.88 KiB/101.25 KiB        
tenacity         

In [3]:
import os
import requests
import re
import time
import pandas as pd

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import VolumeType
from databricks.sdk.service.vectorsearch import (
    EndpointType,
    DeltaSyncVectorIndexSpecRequest,
    EmbeddingSourceColumn,
    PipelineType,
    VectorIndexType,
)
# ============================================================
# 실행 환경 자동 감지
# - Databricks 노트북: "DATABRICKS_RUNTIME_VERSION" 환경변수 존재
# - 로컬: 해당 환경변수 없음
# ============================================================
IS_DATABRICKS = "DATABRICKS_RUNTIME_VERSION" in os.environ
print(f"실행 환경: {'Databricks 노트북' if IS_DATABRICKS else '로컬'}")


실행 환경: 로컬


In [7]:
# ============================================================
# 설정값 (Configuration)
# ============================================================
LLMS_TXT_URL = "https://docs.langchain.com/llms.txt"

CATALOG = "ml"
SCHEMA = "default"
# VOLUME_NAME = "test_volume"

# Delta 테이블 이름 (청킹된 문서 저장용)
TABLE_NAME = f"{CATALOG}.{SCHEMA}.langchain_llms_txt_documents"

# Vector Search 설정
VS_ENDPOINT_NAME = "langchain_docs_vs_endpoint"
VS_INDEX_NAME = f"{CATALOG}.{SCHEMA}.langchain_docs_vs_index"

# 임베딩 모델 (Databricks 제공 모델)
EMBEDDING_MODEL_ENDPOINT = "databricks-qwen3-embedding-0-6b"

# ============================================================
# WorkspaceClient & SparkSession 초기화 — 환경별 분기
# ============================================================
if IS_DATABRICKS:
    # --- [Databricks 노트북 환경] ---
    # spark는 자동으로 제공됨, WorkspaceClient도 기본 인증 사용
    w = WorkspaceClient()
    print(f"Connected to workspace: {w.config.host}")
    print(f"✅ Spark 버전: {spark.version}")
else:
    # --- [로컬 환경] ---
    # databricks auth login 으로 저장된 프로필 사용
    from databricks.connect import DatabricksSession

    # w = WorkspaceClient(profile="dbc-b16a93d0-e9d0")
    w = WorkspaceClient()
    print(f"Connected to workspace: {w.config.host}")

    # Serverless 모드로 연결 (클러스터 불필요)
    spark = DatabricksSession.builder \
        .serverless(True) \
        .getOrCreate()
    print(f"✅ Spark Serverless 연결 완료: {spark.version}")


Connected to workspace: https://dbc-b9376253-212d.cloud.databricks.com
✅ Spark Serverless 연결 완료: 4.0.0


In [5]:
# ============================================================
# Step 1: llms.txt 파일 가져오기 및 URL 파싱
# ============================================================
response = requests.get(LLMS_TXT_URL, timeout=30)
response.raise_for_status()
llms_txt_content = response.text

print(f"llms.txt 파일 크기: {len(llms_txt_content)} bytes")
print("=" * 60)
print(llms_txt_content)
# print(llms_txt_content[:2000])
# print("..." if len(llms_txt_content) > 2000 else "")

# llms.txt에서 마크다운 링크 URL 추출 (형식: [title](url))
# ※ 제목에 [업데이트] 같은 중첩 대괄호가 있으므로 .+? (lazy match) 사용
urls = re.findall(r'\[(.+?)\]\((https?://[^\s\)]+)\)', llms_txt_content)
print(f"\n총 {len(urls)}개의 문서 URL을 발견했습니다.")

# for title, url in urls[:10]:
for title, url in urls:
    print(f"  - {title}: {url}")


llms.txt 파일 크기: 84225 bytes
# Docs by LangChain

## Docs

- [Authenticate](https://docs.langchain.com/api-reference/auth-service-v2/authenticate.md): Get OAuth token or start authentication flow if needed.
- [Check Oauth Token Exists](https://docs.langchain.com/api-reference/auth-service-v2/check-oauth-token-exists.md): Return whether the current user has any tokens for a given provider (across agents).
- [Check Workspace Slack Tokens Exist](https://docs.langchain.com/api-reference/auth-service-v2/check-workspace-slack-tokens-exist.md): Check if the workspace has any Slack tokens.
- [Create Mcp Oauth Provider](https://docs.langchain.com/api-reference/auth-service-v2/create-mcp-oauth-provider.md): Create an OAuth provider via MCP auto-discovery.
- [Create Oauth Provider](https://docs.langchain.com/api-reference/auth-service-v2/create-oauth-provider.md): Create a new OAuth provider manually.
- [Delete Oauth Provider](https://docs.langchain.com/api-reference/auth-service-v2/delete-oauth-p

In [6]:
# ============================================================
# Step 2: 각 URL에서 콘텐츠를 가져와 문서 단위로 저장
# ※ md 문서 하나를 통째로 하나의 청크로 사용 (Splitter 미사용 / 문서 단위로 임베딩)
# ※ md 포맷에 따라 Markdown 청크 적용 가능
# ============================================================
def fetch_iframe_contents(md_content: str) -> str:
    """md 본문에서 <iframe src="..."> 를 찾아 HTML을 가져와 하단에 덧붙인다."""
    iframe_srcs = re.findall(r'<iframe[^>]+src=["\']([^"\']+)["\']', md_content, re.IGNORECASE)
    if not iframe_srcs:
        return md_content

    extra_parts = []
    for src_url in iframe_srcs:
        try:
            r = requests.get(src_url, timeout=30)
            r.raise_for_status()
            extra_parts.append(f"\n\n<!-- iframe content from: {src_url} -->\n{r.text}")
            print(f"      ↳ iframe 파싱: {src_url} ({len(r.text)} chars)")
        except Exception as e:
            print(f"      ↳ iframe 실패: {src_url} — {e}")

    return md_content + "".join(extra_parts)


documents = []

for doc_id, (title, url) in enumerate(urls):
    try:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
        content = resp.text

        # iframe이 있으면 해당 HTML을 가져와 본문에 추가
        content = fetch_iframe_contents(content)

        documents.append({
            "id": doc_id,
            "title": title,
            "url": url,
            "chunk_index": 0,
            "content": content,
        })

        print(f"✅ [{title}] → {len(content)} chars")
    except Exception as e:
        print(f"❌ [{title}] 실패: {e}")

print(f"\n총 {len(documents)}개의 문서를 가져왔습니다.")

# DataFrame으로 변환하여 확인
df = pd.DataFrame(documents)
df.head()


✅ [Authenticate] → 4869 chars
✅ [Check Oauth Token Exists] → 3753 chars
✅ [Check Workspace Slack Tokens Exist] → 2644 chars
✅ [Create Mcp Oauth Provider] → 5597 chars
✅ [Create Oauth Provider] → 6241 chars
✅ [Delete Oauth Provider] → 3195 chars
✅ [Delete Oauth Tokens For User] → 3429 chars
✅ [Get Oauth Provider] → 5119 chars
✅ [List Oauth Providers] → 4095 chars
✅ [Oauth Callback] → 3609 chars
✅ [Oauth Callback Get] → 4491 chars
✅ [Oauth Setup Callback] → 5507 chars
✅ [Revoke All Slack Tokens For Workspace] → 2735 chars
✅ [Update Oauth Provider] → 6228 chars
✅ [Wait For Auth Completion] → 4190 chars
✅ [Create Deployment] → 51770 chars
✅ [Delete Deployment] → 3675 chars
✅ [Delete Deployments] → 3763 chars
✅ [Get Deployment] → 33565 chars
✅ [Get Revision] → 7898 chars
✅ [List Deployments] → 36996 chars
✅ [List Revisions] → 10120 chars
✅ [Patch Deployment] → 51575 chars
✅ [Redeploy Revision] → 34041 chars
✅ [List GitHub Integrations] → 3230 chars
✅ [List GitHub Repositories] → 5281 chars


,id,title,url,chunk_index,content
0,0,Authenticate,https://docs.langchain.com/api-reference/auth-...,0,> ## Documentation Index\n> Fetch the complete...
1,1,Check Oauth Token Exists,https://docs.langchain.com/api-reference/auth-...,0,> ## Documentation Index\n> Fetch the complete...
2,2,Check Workspace Slack Tokens Exist,https://docs.langchain.com/api-reference/auth-...,0,> ## Documentation Index\n> Fetch the complete...
3,3,Create Mcp Oauth Provider,https://docs.langchain.com/api-reference/auth-...,0,> ## Documentation Index\n> Fetch the complete...
4,4,Create Oauth Provider,https://docs.langchain.com/api-reference/auth-...,0,> ## Documentation Index\n> Fetch the complete...


In [8]:
# ============================================================
# Step 3: Unity Catalog에 스키마 생성 및 Delta 테이블 저장
# ※ 로컬: databricks-connect를 통해 원격 클러스터에서 실행됨
# ※ Databricks 노트북: 내장 spark 세션으로 직접 실행
# ============================================================

# 스키마가 없으면 생성
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
print(f"✅ Schema 확인/생성 완료: {CATALOG}.{SCHEMA}")

# Pandas DataFrame → Spark DataFrame → Delta 테이블로 저장
spark_df = spark.createDataFrame(df)

# Delta 테이블 저장
spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(TABLE_NAME)

# Change Data Feed 활성화 (Vector Search Delta Sync에 필요)
spark.sql(f"ALTER TABLE {TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

print(f"✅ Delta 테이블 저장 완료: {TABLE_NAME}")
print(f"   총 {spark.table(TABLE_NAME).count()} rows")
spark.table(TABLE_NAME).show(5, truncate=80)


✅ Schema 확인/생성 완료: ml.default
✅ Delta 테이블 저장 완료: ml.default.langchain_llms_txt_documents
   총 691 rows
+---+----------------------------------+--------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------+
| id|                             title|                                                                             url|chunk_index|                                                                         content|
+---+----------------------------------+--------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------+
|  0|                      Authenticate|        https://docs.langchain.com/api-reference/auth-service-v2/authenticate.md|          0|> ## Documentation Index\n> Fetch the complete documentation index at: https:...|
|  1|          Check Oauth Token Exis

In [9]:
# ============================================================
# Step 4: Vector Search Endpoint 생성 (이미 있으면 건너뜀)
# ============================================================
from databricks.sdk.service.vectorsearch import EndpointStatusState

# 기존 엔드포인트 확인
existing_endpoints = [ep.name for ep in w.vector_search_endpoints.list_endpoints()]

if VS_ENDPOINT_NAME in existing_endpoints:
    print(f"✅ Vector Search Endpoint 이미 존재: {VS_ENDPOINT_NAME}")
else:
    print(f"🔄 Vector Search Endpoint 생성 중: {VS_ENDPOINT_NAME}")
    w.vector_search_endpoints.create_endpoint(
        name=VS_ENDPOINT_NAME,
        endpoint_type=EndpointType.STANDARD,
    )

# Endpoint가 ONLINE 상태가 될 때까지 대기
for i in range(60):
    ep = w.vector_search_endpoints.get_endpoint(VS_ENDPOINT_NAME)
    if ep.endpoint_status and ep.endpoint_status.state == EndpointStatusState.ONLINE:
        print(f"✅ Endpoint ONLINE: {VS_ENDPOINT_NAME}")
        break
    print(f"   ⏳ 대기 중... ({i+1}/60) - 상태: {ep.endpoint_status.state if ep.endpoint_status else 'UNKNOWN'}")
    time.sleep(30)
else:
    print("⚠️ Endpoint가 시간 내에 ONLINE 되지 않았습니다. Databricks UI에서 확인하세요.")


🔄 Vector Search Endpoint 생성 중: langchain_docs_vs_endpoint
   ⏳ 대기 중... (1/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (2/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (3/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (4/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (5/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (6/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (7/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (8/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (9/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (10/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (11/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (12/60) - 상태: EndpointStatusState.PROVISIONING
   ⏳ 대기 중... (13/60) - 상태: EndpointStatusState.PROVISIONING
✅ Endpoint ONLINE: langchain_docs_vs_endpoint


In [11]:
# ============================================================
# Step 5: Vector Search Index 생성 및 Delta Sync
# ============================================================

# 기존 인덱스 확인 — get_index로 직접 조회 (list_indexes는 필터 누락 가능)
try:
    existing_idx = w.vector_search_indexes.get_index(VS_INDEX_NAME)
    print(f"⚠️ 기존 인덱스 발견 — 삭제 후 재생성: {VS_INDEX_NAME}")
    w.vector_search_indexes.delete_index(VS_INDEX_NAME)
    # 삭제 완료 대기
    for i in range(30):
        try:
            w.vector_search_indexes.get_index(VS_INDEX_NAME)
            print(f"   ⏳ 삭제 대기 중... ({i+1}/30)")
            time.sleep(10)
        except Exception:
            print("   ✅ 삭제 완료")
            break
except Exception:
    print(f"ℹ️ 기존 인덱스 없음 — 새로 생성합니다: {VS_INDEX_NAME}")

print(f"🔄 Vector Search Index 생성 중: {VS_INDEX_NAME}")

# Delta Sync 방식으로 Vector Search Index 생성
# - source_table: Delta 테이블에서 자동으로 동기화
# - embedding_source_columns: content 컬럼에 대해 자동 임베딩 생성
w.vector_search_indexes.create_index(
    name=VS_INDEX_NAME,
    endpoint_name=VS_ENDPOINT_NAME,
    primary_key="id",
    index_type=VectorIndexType.DELTA_SYNC,
    delta_sync_index_spec=DeltaSyncVectorIndexSpecRequest(
        source_table=TABLE_NAME,
        pipeline_type=PipelineType.TRIGGERED,
        embedding_source_columns=[
            EmbeddingSourceColumn(
                name="content",
                embedding_model_endpoint_name=EMBEDDING_MODEL_ENDPOINT,
            )
        ],
    ),
)

print(f"✅ Vector Search Index 생성 요청 완료: {VS_INDEX_NAME}")

# 인덱스가 준비될 때까지 대기
for i in range(60):
    idx = w.vector_search_indexes.get_index(VS_INDEX_NAME)
    status = idx.status
    if status and status.ready:
        print(f"✅ Index 준비 완료: {VS_INDEX_NAME}")
        break
    msg = status.message if status else "UNKNOWN"
    print(f"   ⏳ 인덱스 준비 중... ({i+1}/60) - {msg}")
    time.sleep(30)
else:
    print("⚠️ Index가 시간 내에 준비되지 않았습니다. Databricks UI에서 확인하세요.")


⚠️ 기존 인덱스 발견 — 삭제 후 재생성: ml.default.langchain_docs_vs_index
   ⏳ 삭제 대기 중... (1/30)
   ✅ 삭제 완료
🔄 Vector Search Index 생성 중: ml.default.langchain_docs_vs_index
✅ Vector Search Index 생성 요청 완료: ml.default.langchain_docs_vs_index
   ⏳ 인덱스 준비 중... (1/60) - Delta sync Index creation is pending. Check latest status: https://dbc-b9376253-212d.cloud.databricks.com/explore/data/ml/default/langchain_docs_vs_index
   ⏳ 인덱스 준비 중... (2/60) - Index is currently is in the process of syncing initial data. Check latest status: https://dbc-b9376253-212d.cloud.databricks.com/explore/data/ml/default/langchain_docs_vs_index
   ⏳ 인덱스 준비 중... (3/60) - Index is currently is in the process of syncing initial data. Check latest status: https://dbc-b9376253-212d.cloud.databricks.com/explore/data/ml/default/langchain_docs_vs_index
✅ Index 준비 완료: ml.default.langchain_docs_vs_index


In [12]:
# ============================================================
# Step 6: Vector Search 유사도 검색 (Similarity Search) 예시
# ============================================================

# 검색할 질의 (query)
query_text = "LangChain Agent를 만드는 방법"

# Vector Search Index에서 유사도 검색 수행
results = w.vector_search_indexes.query_index(
    index_name=VS_INDEX_NAME,
    columns=["id", "title", "url", "chunk_index", "content"],
    query_text=query_text,
    num_results=5,
)

# 결과 출력
print(f"🔍 검색 질의: \"{query_text}\"")
print(f"📄 검색 결과: {len(results.result.data_array)}건")
print("=" * 80)

for i, row in enumerate(results.result.data_array):
    # 반환 컬럼: columns에 지정한 5개 + __db_score (유사도 점수) 자동 추가
    doc_id, title, url, chunk_index, content, score = row
    print(f"\n--- [{i+1}] {title} (chunk #{chunk_index}) | 유사도: {score:.4f} ---")
    print(f"    URL: {url}")
    # 콘텐츠 미리보기 (200자)
    preview = content[:200].replace("\n", " ")
    # print(f"    내용: {preview}...")
    print(f"    내용: {content}...")
    print()


🔍 검색 질의: "LangChain Agent를 만드는 방법"
📄 검색 결과: 5건

--- [1] Agents (chunk #0.0) | 유사도: 0.7405 ---
    URL: https://docs.langchain.com/oss/python/langchain/agents.md
    내용: > ## Documentation Index
> Fetch the complete documentation index at: https://docs.langchain.com/llms.txt
> Use this file to discover all available pages before exploring further.

# Agents

Agents combine language models with [tools](/oss/python/langchain/tools) to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.

[`create_agent`](https://reference.langchain.com/python/langchain/agents/#langchain.agents.create_agent) provides a production-ready agent implementation.

[An LLM Agent runs tools in a loop to achieve a goal](https://simonwillison.net/2025/Sep/18/agents/).
An agent runs until a stop condition is met - i.e., when the model emits a final output or an iteration limit is reached.

```mermaid  theme={null}
%%{
  init: {
    "fontFamily": "monospace",
 

In [13]:
# ============================================================
# Agent 패키지 설치
# - langchain >= 1.0: create_agent (새 표준 Agent API)
# - databricks-langchain: ChatDatabricks, DatabricksVectorSearch
# - langchain-mcp-adapters: MCP 서버 도구 로드 (선택)
# ============================================================

# --- [로컬 환경] ---
!uv add "langchain>=1.0" databricks-langchain

# --- [Databricks 노트북 환경] ---
# %pip install "langchain>=1.0" databricks-langchain 
# dbutils.library.restartPython()


Resolved 179 packages in 958ms                                       
⠙ pyarrow==22.0.0                                                               Uninstalled 1 package in 64ms
Installed 72 packages in 950ms                              
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.3
 + aiohttp-retry==2.9.1
 + aiosignal==1.4.0
 + alembic==1.18.3
 + attrs==25.4.0
 + blinker==1.9.0
 + contourpy==1.3.3
 + cycler==0.12.1
 + databricks-ai-bridge==0.13.0
 + databricks-langchain==0.14.0
 + databricks-mcp==0.7.0
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + docker==7.1.0
 + flask==3.1.2
 + flask-cors==6.0.2
 + fonttools==4.61.1
 + frozenlist==1.8.0
 + graphene==3.4.3
 + graphql-core==3.2.7
 + graphql-relay==3.2.0
 + gunicorn==23.0.0
 + httpx-sse==0.4.3
 + huey==2.6.0
 + itsdangerous==2.2.0
 + jinja2==3.1.6
 + jiter==0.13.0
 + joblib==1.5.3
 + jsonschema==4.26.0
 + jsonschema-specifications==2025.9.1
 + kiwisolver==1.4.9
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-

In [14]:
# ============================================================
# Step 7: LangChain v1 Agent 설정
# - ChatDatabricks: Databricks Foundation Model 엔드포인트
# - DatabricksVectorSearch: 기존 Vector Search Index를 Retriever로 사용
# - create_agent: LangChain 1.0 새 표준 Agent API
#   (기존 langgraph.prebuilt.create_react_agent 대체)
# ============================================================
from databricks.sdk import WorkspaceClient
from databricks_langchain import ChatDatabricks
from langchain_core.tools import tool
from langchain.agents import create_agent

# 1) Chat Model — Databricks Foundation Model 엔드포인트
#    ※ 사용 가능한 모델: Databricks UI > Serving 에서 확인
LLM_ENDPOINT = "databricks-claude-haiku-4-5"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)
w = WorkspaceClient()

# 2) Vector Search Tool — w.vector_search_indexes를 직접 사용
@tool
def search_langchain_docs(query: str) -> str:
    """LangChain 공식 문서에서 관련 정보를 검색합니다.
    LangChain, LangGraph, LangSmith 사용법, Agent 구축, 도구 연동, RAG, 스트리밍 등의 질문에 사용하세요."""
    
    results = w.vector_search_indexes.query_index(
        index_name=VS_INDEX_NAME,
        columns=["title", "url", "content"],
        query_text=query,
        num_results=3,
    )
    if not results.result.data_array:
        return "관련 문서를 찾지 못했습니다."
    return "\n\n---\n\n".join(
        f"📄 [{title}]({url})\n{content[:2000]}"
        for title, url, content, _score in results.result.data_array
    )

# 4) LangChain v1 create_agent — 새 표준 Agent 생성
SYSTEM_PROMPT = (
    "당신은 LangChain 공식 문서 전문 어시스턴트입니다.\n"
    "사용자의 질문에 답하기 위해 search_langchain_docs 도구로 관련 문서를 검색하세요.\n"
    "검색된 문서 내용을 바탕으로 정확하고 친절하게 한국어로 답변해주세요.\n"
    "답변에는 관련 문서의 URL도 포함해주세요.\n"
    "검색 결과에 없는 내용은 추측하지 말고, 문서에서 확인할 수 없다고 안내해주세요.\n"
    "마지막에 문서 출처를 표시해주세요. 마크다운 URL은 뒤에 .md를 제외한 원본 URL로 나타내주세요. 예를 들면 다음과 같습니다.\n"
    "예시) [원본] https://docs.langchain.com/oss/python/langchain/quickstart, [Markdown] https://docs.langchain.com/oss/python/langchain/quickstart.md 이렇게 구성됩니다.\n"
    "     출처는 [원본] https://docs.langchain.com/oss/python/langchain/quickstart 을 보여주세요."
)

agent = create_agent(
    model=llm,
    tools=[search_langchain_docs],
    system_prompt=SYSTEM_PROMPT,
)

print("✅ Agent 생성 완료 (LangChain v1 create_agent)")
print(f"   LLM: {LLM_ENDPOINT}")
print(f"   Vector Search Index: {VS_INDEX_NAME}")
print(f"   Tool: search_langchain_docs")


/Users/donghoon.han/Desktop/Dev/databricks-korea-ps-assets/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Agent 생성 완료 (LangChain v1 create_agent)
   LLM: databricks-claude-haiku-4-5
   Vector Search Index: ml.default.langchain_docs_vs_index
   Tool: search_langchain_docs


In [15]:
# ============================================================
# Step 8: Agent 실행 예시  (stream 모드 — 중간 과정 + 소요시간 출력)
# ============================================================

def ask_agent(question: str):
    """Agent에 질문하고, 중간 Tool 호출·응답과 각 단계 소요시간을 출력합니다."""
    import time as _t

    print(f"🙋 질문: {question}")
    print("=" * 60)

    t_total = _t.time()   # 전체 시작
    t_step = _t.time()    # 단계별 시작
    step = 0

    for chunk in agent.stream(
        {"messages": [{"role": "user", "content": question}]}
    ):
        for node_name, node_output in chunk.items():
            elapsed = _t.time() - t_step
            step += 1
            messages = node_output.get("messages", [])

            for msg in messages:
                kind = type(msg).__name__

                if kind == "AIMessage":
                    if msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"\n⏱️  [{step}] LLM 사고 — {elapsed:.2f}s")
                            print(f"🔧 [Tool 호출] {tc['name']}({tc['args']})")
                    if msg.content:
                        print(f"\n⏱️  [{step}] LLM 응답 — {elapsed:.2f}s")
                        print(f"🤖 답변:\n{msg.content}")

                elif kind == "ToolMessage":
                    preview = msg.content[:300].replace("\n", " ")
                    print(f"\n⏱️  [{step}] Tool 실행 — {elapsed:.2f}s")
                    print(f"   ↩️  [Tool 결과] ({len(msg.content)} chars) {preview}...")

            t_step = _t.time()  # 다음 단계 타이머 리셋

    total_elapsed = _t.time() - t_total
    print("=" * 60)
    print(f"⏱️  총 소요시간: {total_elapsed:.2f}s")


# 테스트 질문
ask_agent("LangChain에서 Agent를 만드는 방법을 알려주세요.")


🙋 질문: LangChain에서 Agent를 만드는 방법을 알려주세요.

⏱️  [1] LLM 사고 — 2.58s
🔧 [Tool 호출] search_langchain_docs({'query': 'Agent 만드는 방법 구축'})

⏱️  [1] LLM 사고 — 2.58s
🔧 [Tool 호출] search_langchain_docs({'query': 'Agent 기본 사용법 튜토리얼'})

⏱️  [1] LLM 응답 — 2.58s
🤖 답변:
LangChain에서 Agent를 만드는 방법에 대해 검색해드리겠습니다.

⏱️  [2] Tool 실행 — 0.89s
   ↩️  [Tool 결과] (6247 chars) 📄 [Integrations](https://docs.langchain.com/langsmith/integrations.md) > ## Documentation Index > Fetch the complete documentation index at: https://docs.langchain.com/llms.txt > Use this file to discover all available pages before exploring further.  # Integrations  [LangSmith](https://smith.langch...

⏱️  [3] Tool 실행 — 0.02s
   ↩️  [Tool 결과] (6282 chars) 📄 [Create Assistant](https://docs.langchain.com/langsmith/agent-server-api/assistants/create-assistant.md) > ## Documentation Index > Fetch the complete documentation index at: https://docs.langchain.com/llms.txt > Use this file to discover all available pages before exploring further.  # Create 

In [16]:
# ============================================================
# 추가 질문 예시 — 원하는 질문으로 바꿔서 실행하세요
# ============================================================

# ask_agent("LangGraph와 LangChain의 차이점은 무엇인가요?")
# ask_agent("RAG 에이전트를 구축하는 방법을 알려주세요.")
# ask_agent("LangChain에서 도구(Tool)를 정의하는 방법은?")
# ask_agent("LangSmith로 에이전트를 모니터링하려면 어떻게 하나요?")
ask_agent("LangChain에서 멀티 에이전트 시스템은 어떻게 구성하나요?")


🙋 질문: LangChain에서 멀티 에이전트 시스템은 어떻게 구성하나요?

⏱️  [1] LLM 사고 — 1.77s
🔧 [Tool 호출] search_langchain_docs({'query': '멀티 에이전트 시스템 multi-agent'})

⏱️  [1] LLM 사고 — 1.77s
🔧 [Tool 호출] search_langchain_docs({'query': '에이전트 구성 Agent architecture'})

⏱️  [1] LLM 사고 — 1.77s
🔧 [Tool 호출] search_langchain_docs({'query': 'LangGraph 에이전트 coordination'})

⏱️  [1] LLM 응답 — 1.77s
🤖 답변:
LangChain에서 멀티 에이전트 시스템 구성에 대해 검색해드리겠습니다.

⏱️  [2] Tool 실행 — 0.45s
   ↩️  [Tool 결과] (6329 chars) 📄 [Multi-agent](https://docs.langchain.com/oss/javascript/langchain/multi-agent/index.md) > ## Documentation Index > Fetch the complete documentation index at: https://docs.langchain.com/llms.txt > Use this file to discover all available pages before exploring further.  # Multi-agent  Multi-agent sy...

⏱️  [3] Tool 실행 — 0.00s
   ↩️  [Tool 결과] (6260 chars) 📄 [Multi-agent](https://docs.langchain.com/oss/javascript/langchain/multi-agent/index.md) > ## Documentation Index > Fetch the complete documentation index at: https://docs.lang